### Assignment 1 

In [1]:
import numpy as np

In [2]:
with open('diabetes_scale.txt', 'r') as file:
    lines = file.readlines()

# Parse the data into features and labels
X = []
y = []
max_features = 0  
for line in lines:
    parts = line.strip().split(' ')
    label = float(parts[0])
    features = [float(part.split(':')[1]) for part in parts[1:]]
    X.append(features)
    y.append(label)
    max_features = max(max_features, len(features))

for i in range(len(X)):
    while len(X[i]) < max_features:
        X[i].append(0.0)

X = np.array(X)
y = np.array(y)


In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

import torch
from torch.utils.data import Dataset, DataLoader

# Create a custom dataset
class DiabetesDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.FloatTensor(features)
        self.labels = torch.FloatTensor(labels)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# Create datasets and data loaders
train_dataset = DiabetesDataset(X_train, y_train)
test_dataset = DiabetesDataset(X_test, y_test)

batch_size = 64  
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [4]:
import torch

In [5]:
import torch.nn as nn
import torch.nn.functional as F

class Perceptron(nn.Module):
    def __init__(self, input_size):
        super(Perceptron, self).__init__()
        self.fc = nn.Linear(input_size, 1)  # Fully connected layer

    def forward(self, x):
        out = torch.sigmoid(self.fc(x))  # Sigmoid activation for binary classification
        return out

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print(device)

# Create your model and move it to the selected device
model = Perceptron(input_size=8)
model.to(device)

cpu


Perceptron(
  (fc): Linear(in_features=8, out_features=1, bias=True)
)

In [7]:
import torch.optim as optim

# Define loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
learning_rate = 0.001
model = Perceptron(input_size = 8)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

for inputs, labels in trainloader:
    inputs, labels = inputs.to(device), labels.to(device)

In [8]:
# Assuming you have already created the train_loader
num_epochs = 100

for epoch in range(num_epochs):
    total_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1))  # Labels should have shape (batch_size, 1)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Print the average loss for this epoch
    avg_loss = total_loss / len(trainloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

print('Finished Training')

Epoch [1/100], Loss: 0.7401
Epoch [2/100], Loss: 0.7333
Epoch [3/100], Loss: 0.7389
Epoch [4/100], Loss: 0.7353
Epoch [5/100], Loss: 0.7325
Epoch [6/100], Loss: 0.7338
Epoch [7/100], Loss: 0.7304
Epoch [8/100], Loss: 0.7295
Epoch [9/100], Loss: 0.7275
Epoch [10/100], Loss: 0.7305
Epoch [11/100], Loss: 0.7181
Epoch [12/100], Loss: 0.7225
Epoch [13/100], Loss: 0.7215
Epoch [14/100], Loss: 0.7211
Epoch [15/100], Loss: 0.7214
Epoch [16/100], Loss: 0.7185
Epoch [17/100], Loss: 0.7146
Epoch [18/100], Loss: 0.7141
Epoch [19/100], Loss: 0.7145
Epoch [20/100], Loss: 0.7093
Epoch [21/100], Loss: 0.7087
Epoch [22/100], Loss: 0.7076
Epoch [23/100], Loss: 0.7062
Epoch [24/100], Loss: 0.7036
Epoch [25/100], Loss: 0.7033
Epoch [26/100], Loss: 0.7018
Epoch [27/100], Loss: 0.7022
Epoch [28/100], Loss: 0.6997
Epoch [29/100], Loss: 0.7007
Epoch [30/100], Loss: 0.6972
Epoch [31/100], Loss: 0.6990
Epoch [32/100], Loss: 0.6906
Epoch [33/100], Loss: 0.6942
Epoch [34/100], Loss: 0.6946
Epoch [35/100], Loss: 0

In [9]:

model = Perceptron(input_size = 8)  

model.eval()

correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in testloader:
        outputs = model(inputs)  
        predicted_labels = torch.sign(outputs)
        
        total += labels.size(0)
        correct += (predicted_labels == labels).sum().item()

print('Accuracy: %d %%' % (
    100 * correct / total))

Accuracy: 3880 %
